#廣告標題測試 by LSI模型  
##以下純為分析，與選舉政治無關

In [5]:
import gensim
from gensim import corpora,models,similarities,utils
import jieba
import jieba.posseg as pseg
import csv
import pandas as pd

原廣告內容為  
廣告標題:正義召喚善良的你!11/24站出來  
廣告內文:一個人的善良無從自保  
廣告描述:新北市長選舉-侯友宜  
由於標題幾乎沒有關鍵字，故將所有內容皆放進去，並增加了一些當下比較流行的政治關鍵字

In [2]:
ad_title = '一個人的善良無從自保 正義召喚善良的你!11/24站出來 新北市長選舉-侯友宜 政治 韓國瑜 柯P 柯文哲 蘇貞昌 丁守中 姚文智 藍營 綠營 國民黨 民進黨 館長 高家瑜 陳其邁 賴清德 陳菊 選戰'

In [3]:
#載入停用字
with open('/Users/jack/Desktop/LDA/stopword.txt', encoding='utf-8') as f:
    stop_word_list = f.readlines()
    
stop_word_list = [x.strip() for x in stop_word_list]

In [ ]:
#分析部分
analysis_list = [ad_title, data_title]
        lsi_sims_result = []
        texts_list = []
        for i in analysis_list:
            tokens = jieba.cut(i,cut_all=False,HMM=True)
            stopped_tokens = [i for i in tokens if not i in stop_word_list]
            texts_list.append(stopped_tokens)    

            # 重建tfidf model
            dictionary = corpora.Dictionary(texts_list)
            corpus = [dictionary.doc2bow(text) for text in texts_list]
            tfidfModel = models.TfidfModel(corpus)
            # 轉為向量表示
            tfidfVectors = tfidfModel[corpus]

            #建立lsi model
            lsi_model = models.LsiModel(tfidfVectors, id2word=dictionary, num_topics=1)

            # 建立索引
            indexLSI = similarities.MatrixSimilarity(lsi_model[corpus])

            sims = indexLSI[lsi_model[dictionary.doc2bow(texts for texts in data_title)]]
            

推播條件：若相關性大於0.8則推播，若相關性未達則隨機推播(50%機率)

In [ ]:
if sims[0] >= 0.8:
    ad_switch = True
else:
    random_ad = random.uniform(0, 1)
    if random_ad > 0.5:
        ad_switch = True
    else:
        ad_switch = False

#結果：

In [9]:
#載入整理好之記錄
result = pd.read_csv('analysis_record.csv')
result.head()

,新聞標題,相關性
0,錯愕！35萬人假帳號明星也被唬 林依晨本人IG是這個,0.0
1,NBA／豪神！林書豪狂奪26分本季新高 老鷹依舊慢飛,0.0
2,以房養老已近3千件 公股行庫占9成 建議貸款期限至50年,0.0
3,管中閔透露選後考量：我其實已經看到選舉結果,0.0
4,神對比！館長助選柯P、姚文智嗆高家瑜 人渣文本：草包,0.0


In [14]:
#有過標準之標題
result[result['相關性'] >0.8]

,新聞標題,相關性
57,「神秘嘉賓」周美青站台？馬英九：她今年都不參加活動,1.0
60,影／與人民站在一起 韓國瑜選前之夜不邀大咖上台造勢,1.0
63,韓國瑜因「護唇膏黏黏的」辯論失常？ 余天傻眼：牽拖啦,1.0
89,配合選舉！台鐵周末開放花東新自強號站票,1.0
101,NOW晚報／姚文智當眾公審 高嘉瑜「8字回應」網友心疼,1.0
115,兵分三路 韓國瑜與妻女分身掃街拉票,1.0
140,韓國瑜選前感性直播！「神秘嘉賓」現身 瞬間擠破7萬粉,1.0
146,房市／大湳轉運站將動工 活絡桃園八德房市,1.0
149,影／撞臉韓星…高嘉瑜13年前「青澀片」曝光 腰身被讚翻,1.0
166,林夕批評韓國瑜政見空泛 韓冰化身DJ點播一首「紅豆」,1.0


In [15]:
#計算過標準率
len(result[result['相關性'] >0.8])/len(result)

0.057692307692307696

##約5~6%之新聞通過LSI模型篩選  
最終獲得4367之點擊數(約一天時間)